# トラブルシューティング


## 準備

本章のコマンドを実行するための設定を行います。  

セットアップ済みの環境のうちコマンドを送る先であるCoordinating(Client) Nodeのホストアドレスを、次のセルに記入して実行し、保存してください。

In [36]:
%env ES_HOST=XXX.XXX.XXX.232:9200

env: ES_HOST=XXX.XXX.XXX.232:9200


## トラブル発生時の情報収集の流れ

本Notebookでは、運用中の一般的なトラブル発生時の情報収集の流れについて説明します。  
上から順に試し、原因究明に必要な情報が得られなければ次の手順に進んでください。  

以下の流れで説明します。
1. Clusterの状態を確認
1. Elasticsearchのログを確認 
1. Slowログを確認
1. Elasticsearchのプロセス状態を確認
1. JVMの状態を確認

上記2,3,4,5は、情報を取得したいサーバへログインして作業してください。

### Clusterの状態を確認

障害発生時にはClusterの状態を確認して下さい。  
"status"が"green"でない場合、障害が発生している可能性があります。  
クラスタ状態についての詳細は、Elasticsearch Referenceの[Cluster Health](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/cluster-health.html)を参照してください。

In [5]:
%%bash
curl -XGET "http://$ES_HOST/_cluster/health?pretty"

{
  "cluster_name" : "cluster-name",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 5,
  "number_of_data_nodes" : 3,
  "active_primary_shards" : 65,
  "active_shards" : 130,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   468  100   468    0     0  50101      0 --:--:-- --:--:-- --:--:-- 52000


### Elasticsearchログを確認

ERROR,WARNメッセージ等が出力されていないか確認し、内容に応じて対処してください。  
ログはデフォルトでは次の場所に出力されています。  

/var/log/elasticsearch/***Clustername***.log  
太字は設定したクラスタ名になります。

以下のサンプルログは、Elasticsearch起動時にBootstrapCheckでエラーになっています。  
詳細はElasticsearch Referenceの[Maximum map count check](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/_maximum_map_count_check.html)を参照ください。

### Slowログを確認

クエリ結果が返ってこない場合、slowログから実行に時間がかかっていないか確認してください。  
ログファイルはlog4j2.propertiesに設定した場所に出力されています。

/var/log/elasticsearch/elasticsearch_index_search_slowlog.log

/var/log/elasticsearch/elasticsearch_index_indexing_slowlog.log

slowログには実行したクエリが記載されています。  
このクエリを解析するためにはProfile APIを使用します。  
例として、Profile APIのコマンドは次の通りです。

In [76]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.01/_search?pretty" -d @- << EOF
{
  "profile": true,
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 6,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 0,
    "max_score" : null,
    "hits" : [ ]
  },
  "profile" : {
    "shards" : [
      {
        "id" : "[sjROHXMsTxKfXSdoCbpRSQ][meteorological-data-2015.01][1]",
        "searches" : [
          {
            "query" : [
              {
                "type" : "TermQuery",
                "description" : "location:tokyo",
                "time" : "0.1589970000ms",
                "breakdown" : {
                  "score" : 0,
                  "build_scorer_count" : 0,
                  "match_count" : 0,
                  "create_weight" : 158996,
                  "next_doc" : 0,
                  "match" : 0,
                  "create_weight_count" : 1,
                  "next_doc_count" : 0,
                  "score_count" : 0,
                  "build_scorer" : 0,
                  "advance" : 0,
                  "advance_count" : 0
  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5994  100  5921  100    73   476k   6020 --:--:-- --:--:-- --:--:--  525k


Profile APIの詳細は、Elasticsearch Referenceの[Profile API](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/search-profile.html)を参照してください。

ただし、Profileの結果は時間が立つと変化してしまうため注意が必要です。  
これは、データがどのshardに入るのかは、データのハッシュ値によって変化するため、  
クラスタ状況によってはデータの再配置が起きて、shard間でデータの移動が行われる可能性があるためです。  
（例：ある特定ノードで残容量がwater markを超過したためにshardの移動が発生した。など)  

shard数の決め方については[01_Setup](01_Setup.ipynb#Shard数を決める)を参照してください。

### Elasticsearchのプロセス状態を確認

プロセスが起動していること、%CPU,%MEMの項目からサーバリソースを消費しすぎていないことを確認してください。

### JVMの状態を確認

GCログからメモリの使用状況を確認してください。  
ログはjvm.optionsに設定した場所に出力されています。

/var/log/elasticsearch/gcssd.log

jstatからFull GCが頻発していないか確認してください。  
E列が100%近くになっていたり、FGCが頻発していた場合、ヒープサイズを見直す必要があります。

## Queryの処理が遅い

### タスクが実行中の場合

task APIを利用し、クラスタ内のノードで実行中のタスクを確認することができます。  
APIの詳細はElasticsearch Referenceの[Task Management API](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/tasks.html)を参照してください。


In [11]:
%%bash
curl -XGET "http://$ES_HOST/_tasks?pretty"

{
  "nodes" : {
    "JjtDV9M6SBeUQtcAfgZaBQ" : {
      "name" : "node-3",
      "transport_address" : "XXX.XXX.XXX.233:9300",
      "host" : "XXX.XXX.XXX.233",
      "ip" : "XXX.XXX.XXX.233:9300",
      "roles" : [
        "master",
        "ingest"
      ],
      "tasks" : {
        "JjtDV9M6SBeUQtcAfgZaBQ:5596" : {
          "node" : "JjtDV9M6SBeUQtcAfgZaBQ",
          "id" : 5596,
          "type" : "netty",
          "action" : "cluster:monitor/tasks/lists[n]",
          "start_time_in_millis" : 1481515589212,
          "running_time_in_nanos" : 194896,
          "cancellable" : false,
          "parent_task_id" : "nuc7d3LiTFamTIsEQgo--w:1545"
        }
      }
    },
    "DpijOSvlStK58xOfjvlWhg" : {
      "name" : "node-4",
      "transport_address" : "XXX.XXX.XXX.234:9300",
      "host" : "XXX.XXX.XXX.234",
      "ip" : "XXX.XXX.XXX.234:9300",
      "roles" : [
        "data",
        "ingest"
      ],
      "tasks" : {
        "DpijOSvlStK58xOfjvlWhg:1798" : {
          "node" :

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3624  100  3624    0     0   238k      0 --:--:-- --:--:-- --:--:--  252k


indexing作業中はクエリの応答に時間がかかる場合があります。詳しくはElasticsearch Referenceの[Indexing Performance Tips](https://www.elastic.co/guide/en/elasticsearch/guide/2.x/indexing-performance.html)を参照してください。

### 検索期間が長すぎる場合

クエリの検索対象期間が長い場合、応答が遅くなる可能性があります。

例として、日ごとにindexを区切ってドキュメントを登録している場合  
クエリの検索期間を短くして実行してください。

検索期間を「meteorological-data-2015.01」「meteorological-data-2015.02」に絞ってドキュメントを検索するコマンドは次の通りです。

In [12]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-2015.01,meteorological-data-2015.02/_search?pretty" -d @- << EOF
{
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 128,
  "timed_out" : false,
  "_shards" : {
    "total" : 10,
    "successful" : 10,
    "failed" : 0
  },
  "hits" : {
    "total" : 28,
    "max_score" : 0.18232156,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.02",
        "_type" : "logs",
        "_id" : "tokyo_15-02-02",
        "_score" : 0.18232156,
        "_source" : {
          "date" : "15-02-02",
          "precipitation_max_10min" : null,
          "wind_speed_avg" : "4.7",
          "snowfall" : null,
          "temperature_max" : "9.6",
          "precipitation_max_hour" : null,
          "wind_direction" : "西",
          "information_daytime" : "晴",
          "precipitation_day" : null,
          "@timestamp" : "2015-02-01T15:00:00.000Z",
          "wind_speed_max" : "9.1",
          "sunshine_duration" : "9.6",
          "humidity_avg" : "34.0",
          "humidity_min" : "22.0",
          "temperature_avg" : "4.7",
          "temperature_min" : "1.0",
          "@version" : "1",
  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11444  100 11389  100    55  69642    336 --:--:-- --:--:-- --:--:-- 69871


* 次のコマンドのように長い期間のクエリを実行したい場合、indexの区切り方を変える必要があります

In [39]:
%%bash
curl -XGET "http://$ES_HOST/meteorological-data-*/_search?pretty" -d @- << EOF
{
  "query": {
    "term" : { "location" : "tokyo" } 
  }
}
EOF

{
  "took" : 14,
  "timed_out" : false,
  "_shards" : {
    "total" : 70,
    "successful" : 70,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 0.2876821,
    "hits" : [
      {
        "_index" : "meteorological-data-2014.12",
        "_type" : "logs",
        "_id" : "tokyo_15-01-01",
        "_score" : 0.2876821,
        "_source" : {
          "date" : "15-01-01",
          "wind_speed_avg" : "4.4",
          "snowfall" : null,
          "temperature_max" : "8.2",
          "precipitation_day" : "0",
          "wind_speed_max" : "9.4",
          "sunshine_duration" : "1.3",
          "temperature_avg" : "3.7",
          "temperature_min" : "0.7",
          "@version" : "1",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "precipitation_max_10min" : "0",
          "precipitation_max_hour" : "0",
          "wind_direction" : "西",
          "information_daytime" : "時々晴一時雪",
          "@timestamp" : "2014-12-31T15:00

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11439  100 11384  100    55   403k   1996 --:--:-- --:--:-- --:--:--  411k


## ディスクフルを防ぐ仕組み

ディスク使用率が100%にならないように、各ノードのディスク領域に応じたシャード割り当ての機能があります。  
例.  
ディスク使用率が80%を超えた場合、そのノードには新しいシャードを割り当てなくし、  
ディスクの空き領域が50GBあった場合、シャードを再配置する設定は次の通り

elasticsearch.ymlに設定

water markを超過したことを示すメッセージは次のようなものです。

詳細はElasticsearch Referenceの[Disk-based Shard Allocation](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/disk-allocator.html)を参照してください。

## タイムゾーンについての注意

Elasticsearchの内部ではデータの時間をUTCで持っています。  
例えば、インダックス名に"some_index-2015.01.01"のような日付を使用していた場合、日本時間で1月1日8時のデータは"some_index-2014.12.31"の23時のデータとして持ちます。9時間ずれた結果、前日のインデックスに登録されます。インデックスを指定して検索する場合には注意が必要です。  
  
データ投入時に使用したいタイムゾーンに合わせる必要があります。  
データ投入後にタイムゾーンを変更する場合、再投入が必要となります。

タイムゾーンを指定したクエリについてはElasticsearch Referenceの[Time Zone](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/search-aggregations-bucket-datehistogram-aggregation.html#_time_zone)を参照してください。